In [1]:
# this script reads an input file (csv) with the following columns: gene, predicted_label, confidence, description, annotated
# it also reads a list of genes (one per line) from a csv file with a header "gene"
# the first csv file contains a list of genes and their GO terms annotations in the column "predicted_label"
# First filter the file to keep annotated == "Yes"
# Second, take the list of genes from the second file and calculate an enrichment test for the list of GO terms given in the first file
# The enrichment test is done using the hypergeometric test from scipy.stats.hypergeom
# The output is a csv file with the following columns: predicted_label, pvalue, enrichment
# use pandas to read the csv files
# import the files in the terminal line using the following command:
# python proteinfer_enrich.py input.csv genes.csv

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import hypergeom
from tqdm import tqdm
import multiprocessing as mp

In [10]:
# read the input file
df = pd.read_csv("/Users/danmarti/Documents/MRC_postdoc/My_projects/Ecoli_metabolic_modelling/tables/proteinfer_full.csv", sep=",", header=0)

# filter the input file to keep only annotated == "Yes"
df = df[df.annotated == "Yes"]

# read the list of genes
query_genes = pd.read_csv("/Users/danmarti/Documents/MRC_postdoc/My_projects/Ecoli_metabolic_modelling/tables/gene_result_list.csv", sep=",", header=0)

# to calculate the hypergeometric test, we need the following variables:
# total_genes: total number of genes in the genome
# total_GO_terms: total number of GO terms in the input file
# total_annotated_genes: total number of genes annotated with at least one GO term
# n: number of genes annotated with the GO term
# k: number of genes annotated with the GO term and in the list of genes
# M: total number of genes in the list of genes

# get the list of GO terms
GO_terms = df["predicted_label"].unique()

# get the list of genes
genes_list = query_genes["gene"].unique()

# get the total number of genes
total_genes = len(df['gene'].unique())

# get the total number of GO terms
total_GO_terms = len(GO_terms)

# get the total number of annotated genes
total_annotated_genes = len(df["gene"].unique())

# create a new dataframe to store the results
results = pd.DataFrame(columns=["predicted_label", "pvalue", "enrichment"])

# define a function to calculate the hypergeometric test
def hypergeom_test(GO_term):
    # get the list of genes annotated with the GO term
    GO_term_genes = df[df["predicted_label"] == GO_term]["gene"].unique()
    # get the number of genes annotated with the GO term
    n = len(GO_term_genes)
    # get the number of genes annotated with the GO term and in the list of genes
    k = len(np.intersect1d(GO_term_genes, genes_list))
    # calculate the hypergeometric test
    pvalue = hypergeom.sf(k-1, total_genes, n, total_annotated_genes)
    # calculate the enrichment
    enrichment = k/n
    # return the results
    return pd.Series({"predicted_label": GO_term, "pvalue": pvalue, "enrichment": enrichment})

# calculate the hypergeometric test for each GO term
# use multiprocessing to speed up the process
# use tqdm to show the progress bar
if __name__ == "__main__":
    with mp.Pool(mp.cpu_count()) as pool:
        for i, _ in tqdm(enumerate(pool.imap_unordered(hypergeom_test, GO_terms)), total=len(GO_terms)):
            results = results.append(_, ignore_index=True)


Process SpawnPoolWorker-103:
Process SpawnPoolWorker-102:
Process SpawnPoolWorker-101:
Traceback (most recent call last):
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'hypergeom_test' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/danmarti/miniforge3/envs/pyseer/lib/python3.10/multiproce